In [2]:
import scanpy as sc
import squidpy as sq

import numpy as np

In [3]:
adata = sq.datasets.visium_fluo_adata()
adata

  0%|          | 0.00/242M [00:00<?, ?B/s]

AnnData object with n_obs × n_vars = 2800 × 16562
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)

In [7]:
adata.obsp["spatial_connectivities"].A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
adata

AnnData object with n_obs × n_vars = 2800 × 16562
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial', 'umap', 'spatial_neighbors'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances', 'spatial_connectivities', 'spatial_distances'

In [32]:
import numpy as np
import torch
from scipy import sparse
from torch.utils.data import Dataset

class SpatialAnnDataDataset(Dataset):
    """

       Parameters
       ----------
       adata: `~anndata.AnnData`
            Spatially annotated AnnData object. Adjaceny matrix needs to be stored in adata.obsp["spatial_connectivities"].
    """

    def __init__(self,
                 adata,
                 ):

        if sparse.issparse(adata.X):
            self.X = torch.tensor(adata.X.A)
        else:
            self.X = torch.tensor(adata.X)

        if sparse.issparse(adata.obsp["spatial_connectivities"]):
            self.A = torch.tensor(adata.obsp["spatial_connectivities"].A)
        else:
            self.A = torch.tensor(adata.obsp["spatial_connectivities"]) # output from squidpy.gr.spatial_neighbors

        if not (self.A==self.A.T).all():
            raise ImportError("The input adjacency matrix is not a symmetric matrix.")
        if not np.diag(self.A).sum()==0:
            raise ImportError("The diagonal elements of input adjacency matrix are not all 0.")


    def __getitem__(self, index):
        output = dict()

        output["X"] = self.X[index, :]
        output["A"] = self.A[index, :]

        return output


    def __len__(self):
        return self.X.size(0)


    def __str__(self):
        return self.__name

True

In [20]:
sparse.issparse(adata.obsp["spatial_connectivities"])

True

In [9]:
import torch
from scipy import sparse
from torch.utils.data import Dataset

In [35]:
import numpy as np
import torch
from scipy import sparse
from torch.utils.data import Dataset

class SpatialAnnDataDataset(Dataset):
    """

       Parameters
       ----------
       adata: `~anndata.AnnData`
            Spatially annotated AnnData object. Adjaceny matrix needs to be stored in adata.obsp["spatial_connectivities"].
    """

    def __init__(self,
                 adata,
                 ):

        if sparse.issparse(adata.X):
            self.X = torch.tensor(adata.X.A)
        else:
            self.X = torch.tensor(adata.X)

        if sparse.issparse(adata.obsp["spatial_connectivities"]):
            self.A = torch.tensor(adata.obsp["spatial_connectivities"].A)
        else:
            self.A = torch.tensor(adata.obsp["spatial_connectivities"]) # output from squidpy.gr.spatial_neighbors

        if not (self.A==self.A.T).all():
            raise ImportError("The input adjacency matrix is not a symmetric matrix.")
        if not np.diag(self.A).sum()==0:
            raise ImportError("The diagonal elements of input adjacency matrix are not all 0.")


    def __getitem__(self, index):
        output = dict()

        output["X"] = self.X[index, :]
        output["A"] = self.A[index, :]

        return output


    def __len__(self):
        return self.X.size(0)


    def __str__(self):
        return self.__name

In [51]:
dataset = SpatialAnnDataDataset(adata)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [2000, 800])

In [52]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [53]:
type(train_dataset)

torch.utils.data.dataset.Subset

In [55]:
next(iter(train_dataloader))

{'X': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.4659],
         [0.0000, 0.0000, 0.7449,  ..., 0.0000, 0.0000, 1.1671],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.9113],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2614]]),
 'A': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)}

In [40]:
test_dataset[5:12]

{'X': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.5867],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.5107],
         [0.7439, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.8750],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2222],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.7781],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'A': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)}

In [39]:
len(test_dataset)

2800